In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Hutchinson-Gilford_Progeria_Syndrome/GSE3860'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Comparison of Hutchinson–Gilford Progeria Syndrome fibroblast cell lines to control fibroblast cell lines"
!Series_summary	"Hutchinson–Gilford progeria syndrome (HGPS) is a rare genetic disease with widespread phenotypic features resembling premature aging. HGPS was recently shown to be caused by dominant mutations in the LMNA gene, resulting in the in-frame deletion of 50 amino acids near the carboxyl terminus of the encoded lamin A protein. Children with this disease typically succumb to myocardial infarction or stroke caused by severe atherosclerosis at an average age of 13 years. To elucidate further the molecular"
!Series_summary	"pathogenesis of this disease, we compared the gene expression patterns of three HGPS fibroblast cell lines heterozygous for the LMNA mutation with three normal, age-matched cell lines. We defined a set of 361 genes (1.1% of the approximately 33 000 genes analyzed) that showed at least a 2-fold, statistically signifi

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Initialize Sample_Characteristics_Dictionary
Sample_Characteristics_Dictionary = {
    0: [
        'Cell Line: AG10750, Gender: Male, Age: 9 years', 
        'Line: AG10750, Gender: Male, Age: 9 years', 
        'Line: AG11498, Gender: Male, Age: 14 years', 
        'Cell Line: AG11498, Gender: Male, Age: 14 years', 
        'Cell Line: AG11513, Gender: Female, Age: 8 years', 
        'Cell Line: GM00038C, Gender: Female, Age: 9 years', 
        'Line: GM00038C, Gender: Female, Age: 9 years', 
        'Cell Line: GM0316B, Gender: Male, Age: 12 years', 
        'Cell Line: GM08398C, Gender: Male, Age: 8 years'
    ]
}

# Check if gene expression data is available
is_gene_available = True

# Check data availability and keys in sample characteristics
trait_row = 0 if 'HUTCHINSON-GILFORD PROGERIA SYNDROME' in "!Series_title".upper() else None
age_row = 0 if any('Age' in entry for entry in Sample_Characteristics_Dictionary[0]) else None
gender_row = 0 if any('Gender' in entry for entry in Sample_Characteristics_Dictionary[0]) else None

# Define conversion functions
def extract_value(cell, keyword):
    for field in cell.split(','):
        if keyword in field:
            return field.split(':')[1].strip()
    return None

def convert_trait(cell):
    # Assuming that specific lines are known as HGPS or control based on domain knowledge
    hgps_lines = ['AG10750', 'AG11498', 'GM0316B']  # List of known HGPS cell lines
    value = extract_value(cell, 'Cell Line')
    if value:
        return 1 if value in hgps_lines else 0
    return None

def convert_age(cell):
    value = extract_value(cell, 'Age')
    if value:
        return float(value.split()[0])
    return None

def convert_gender(cell):
    value = extract_value(cell, 'Gender')
    if value:
        return 1 if value.lower() == 'male' else 0
    return None

# Save cohort information
save_cohort_info('GSE3860', './preprocessed/Hutchinson-Gilford_Progeria_Syndrome/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical feature extraction if trait data is available
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Hutchinson-Gilford_Progeria_Syndrome', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Hutchinson-Gilford_Progeria_Syndrome/trait_data/GSE3860.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
